In [10]:
import os
import dicom
import cv2
import csv
import numpy as np
import math
from matplotlib import pyplot as plt

DATASET_BASE = "D:/doc/4/graduation/data"
PATH_raw_info = DATASET_BASE + '/data1.csv'
fname = PATH_raw_info
id = 194
with open(fname, 'r') as csvfile:
    rows = csv.reader(csvfile)
    id = id - 1
    for row in rows:
        if row[0] == str(id):
            name = row[1]
            x = int(row[2])
            y = int(row[3])
            z = int(row[4])
            loc = "D:/doc/4/graduation/data/193"
            
print(os.path.join(loc, 'dicomdir'))
ds = dicom.read_file(os.path.join(loc, 'dicomdir'))
pixel_data = [np.zeros((512, 512)), np.zeros((512, 512))]

for record in ds.DirectoryRecordSequence:
    if record.DirectoryRecordType == "IMAGE":
        # Extract the relative path to the DICOM file
        path = os.path.join(loc, record.ReferencedFileID[0], record.ReferencedFileID[1])
        dcm = dicom.read_file(path)
        if dcm.InstanceNumber == z:
            if "WB" in dcm[0x08, 0x103e].value:
                if "PET" in dcm[0x08, 0x103e].value:
                    pet_spa = dcm.PixelSpacing[0]
                    z = dcm.InstanceNumber
                    pet_len = dcm.Rows
                    slice_num = dcm.NumberOfSlices
                    pixel_data[0][0:pet_len, 0:pet_len] = dcm.pixel_array
                    RIS = dcm[0x54,0x16].value
                    dose = RIS[0][0x18,0x1074].value
                    half_life = RIS[0][0x18,0x1075].value
                    start = RIS[0][0x18,0x1072].value
                    scan = dcm[0x08,0x32].value
                    decay = float(scan) - float(start)
                    decayed_dose = pow(2, decay/half_life)
                    res_slope = dcm[0x28,0x1053].value
                    res_intercept = dcm[0x28,0x1052].value
                    PET_num = dcm.NumberOfSlices
                    pet_test = dcm[0x7fe0, 0x10].value
            if "5mm" in dcm[0x08, 0x103e].value or "CT_WB" in dcm[0x08, 0x103e].value:
                ct_len = dcm.Rows
                z = dcm.InstanceNumber
                ct_spa = dcm.PixelSpacing[0]
                pixel_data[1][0:ct_len, 0:ct_len] = dcm.pixel_array

pet0 = pixel_data[0][0:pet_len, 0:pet_len]
ct = pixel_data[1][0:ct_len, 0:ct_len]
del pixel_data
pet1 = cv2.resize(pet0, dsize=None, fx=pet_spa / ct_spa, fy=pet_spa / ct_spa)
pet_len = pet1.shape[1]
if (pet_len > 511):
    centre = int(pet_len / 2)
    pet2 = pet1[centre - 256:centre + 256, centre - 256:centre + 256]
else:
    len0 = pet_len
    if (len0 % 2 == 0):
        pad = int(256 - len0 / 2)
        pet2 = cv2.copyMakeBorder(pet1, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    else:
        pad = int(256 - len0 / 2)
        pet2 = cv2.copyMakeBorder(pet1, pad - 1, pad, pad, pad - 1, cv2.BORDER_REPLICATE)
pet = pet2
ct = ct

print(type(pet_test))
print(z)
suv_img = (pet * res_slope + res_intercept) * decayed_dose * (dcm[0x10,0x1030].value * 1000 / dose) 
# suv_2 = U / (1000 / Ac * dcm[0x10,0x1030].value * math.exp(-0.693 / T1 * (t0-t1)))
print(np.max(suv_img))
print(np.where(pet == np.max(pet)))
print(np.where(suv_img == np.max(suv_img)))
print(suv_img[280,240])
print(dcm[0x10,0x1030].value)
# 
# plt.figure(figsize=(15, 15))
# plt.subplot(2, 2, 1)
# plt.imshow(pet)
# plt.subplot(2, 2, 2)
# plt.imshow(ct)
# plt.subplot(2, 2, 3)
# plt.imshow(ct, plt.cm.gray)
# plt.imshow(pet, plt.cm.afmhot, alpha=0.5)
# plt.scatter(240, 280, color='', marker='o', edgecolors='w', s=200)
# plt.title(1)
# plt.show()

D:/doc/4/graduation/data/193\dicomdir


<type 'str'>
29
5.3756382810167755
(array([280]), array([240]))
(array([280]), array([240]))
5.3756382810167755
72


In [29]:
import pandas as pd
import numpy as np
import collections
from itertools import repeat

In [8]:
INFO = pd.read_csv('D:/doc/guo/diannei/doc/code_check/invasion/csv/INFO_lidc_0131.csv')
a = list(INFO[INFO['subset'] ==0].index)

In [16]:
subset = [1,2,3,4]
invasion = 'subset'
LABEL_NAME = 'invasion_label'
LABEL = ['AAH', 'AIS', 'MIA', 'IA', 'OTHER']
index = []
for sset in subset:
    index += list(INFO[INFO[invasion] == sset].index)
index = tuple(sorted(index))  # the index in the info
label = np.array([[label == s for label in LABEL] for s in INFO.loc[index, LABEL_NAME]])

In [27]:
def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.Iterable):
            return x
        return tuple(repeat(x, n))

    return parse

_ntuple(3)

<function __main__.parse>

In [35]:
a = _ntuple(3)
print(a("huhy"))

huhy
